# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = '8'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 4 max_iteration = 5 model_type = base *********
n                 iter (since)         4 loss (since)       4 rouge1 (since)   4 rouge1 ema (since) 4 rouge1 (dev) (since)      dt
1                0.000 (0.000)          2.260 (2.260)          0.426 (0.426)          0.426 (0.426)          0.000 (0.000)  7.66 s
2                0.000 (0.000)          2.047 (1.835)          0.464 (0.502)          0.459 (0.492)          0.000 (0.000)  13.9 s
4                0.000 (0.000)          1.949 (1.850)          0.436 (0.409)          0.462 (0.466)          0.000 (0.000)  25.2 s
16               0.000 (0.000)          1.983 (1.987)          0.426 (0.431)          0.435 (0.424)          0.000 (0.000)  1.56 m
32               0.000 (0.000)          1.967 (1.950)          0.438 (0.449)          0.440 (0.444)          0.000 (0.000)  3.01 m
64               0.000 (0.000)          1.902 (1.838)          0.435 (0.433)          0.438 (0.437)          0.000 (0.000)  5.93 m
128             

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = '2'
    os.environ['LEARN_BATCH_SIZE'] = '4'
    os.environ['GRAD_FREE_BATCH_SIZE'] = '16'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()